# HIGH ORDER NOTEBOOK

**Authors : GIRIER Guillaume, DESROCHES Mathieu, RODRIGUES Serafim (Basque Center for Applied Mathematics)**

**Contact : guillaumegirier@gmail.com**

**Objective :**

*The purpose of this notebook is to guide the user through the different functions of this project, as well as to provide some examples to understand how it works.*

**Packages :**

In [1]:
import numpy as np
import cmath

from scipy.stats import norm
from itertools import combinations
from scipy.special import digamma

import dit

**Functions :**

**gaussian_ent_biascorr()** *compute the bias corrector for the entropy estimator based on covariance matrix of gaussian.*

In [2]:
def gaussian_ent_biascorr(N,T):

	"""
	
	INPUTS:
	
	N = Number of dimensions
	T = Sample size
	
	OUTPUTS
	
	biascorr = bias corrector value
	"""
	
	values = np.arange(1, N+1)
	
	return 0.5 * ((N * np.log(2/(T-1))) + np.sum(list(map(lambda n : digamma((T-n)/2), values))) )

*example :*

In [3]:
N = 2
T = 15
print("Bias corrector for the entropy : ", gaussian_ent_biascorr(N,T))

Bias corrector for the entropy :  -0.11306231630611352


**data2gaussian()** *transforms 'data' to Gaussian with mean = 0 and sd = 1 using empirical copulas.*

In [4]:
def data2gaussian (data) :

	"""
	
	INPUTS :
	
	data = T samples x N variables matrix
	
	OUTPUTS :
	
	gaussian_data = T samples x N variables matrix with the gaussian copula transformed data.
	covmat = N x N covariance matrix of gaussian copula transformed data.
	
	"""
	
	T = len(data)
	sort_index = np.argsort(data, axis = 0) # Sort the data and keep the indexes.
	copdata = np.argsort(sort_index, axis = 0) # Sorting sorting indexes
	copdata += 1 # To avoid 0 because of the python indexation
	copdata = copdata/ (T+1) # Normalization.
	gaussian_data = norm.ppf(copdata, 0, 1) # PPF : Probability Density Function  => Gaussian data
	gaussian_data[~np.isfinite(gaussian_data)] = 0 # Removing -Inf
	cov_mat = np.dot(np.transpose(gaussian_data), gaussian_data ) / (T-1) # Covariance matrix
	
	return gaussian_data, cov_mat


*example :*

In [18]:
real_data = np.array([[ 2., 4.], [ 5., 1.]])

gaussian_data, cov_mat = data2gaussian(real_data)


test = np.array([[ 0., 0.], [ 1, 0]])

d = dit.Distribution.from_ndarray(test)
print(dit.multivariate.total_correlation(d))

print("gaussian data : \n", gaussian_data, "\n")
print("Covariance matrix :\n",cov_mat)

0.0
gaussian data : 
 [[-0.4307273  0.4307273]
 [ 0.4307273 -0.4307273]] 

Covariance matrix :
 [[ 0.37105201 -0.37105201]
 [-0.37105201  0.37105201]]


**ent_fun()** *and* **reduce_x()** *are two functions used in soinfo_from_covmat().*

**ent_fun()** *function compute the entropy of multivariate gaussian distribution where x is dimensionality and y is the variables variance of the covariance matrix determinant.*

**reduce_x()** *remove the line and row x in order to obtain a submatrix len(covmat)-1x len(covmat)-1.*

**soinfo_from_covmat()** *computes the 0-information and S-information of gaussian data given their covariance matrix 'covmat'*

In [25]:
def ent_fun (x,y):

	"""
	
	In order to avoid log(0), we replace the returned value as NaN value.
	"""
	
	if (( 2*np.pi*np.exp(1) ) ** x) * y == 0:
	
		return np.nan
	else:
		return 0.5 * cmath.log((( 2*np.pi*np.exp(1) ) ** x) * y)



def reduce_x (x, covmat):

	covmat = np.delete(covmat, x, axis = 0)
	covmat = np.delete(covmat, x, axis = 1)
	
	return covmat



def soinfo_from_covmat (covmat, T):
	
	"""
	
	INPUTS :
	
	covmat = N x N covariance matrix
	T = lenght data
	
	OUPUTS :
	
	oinfo = O - Information
	sinfo = S - Information of the system with covariance matrix 'covmat'.
	
	"""
	
	covmat = np.array(covmat)
	N = len(covmat)
	emp_det = np.linalg.det(covmat) # Determinant
	single_vars = np.diag(covmat) # Variance of single variables (Diagonal matrix values)
	
	### Bias corrector for N, (N-1) and one gaussian variables :
	
	biascorrN = gaussian_ent_biascorr(N, T)
	biascorrNmin1 = gaussian_ent_biascorr(N-1, T)
	biascorr_1 = gaussian_ent_biascorr(1, T)
	
	### Computing estimated measures for multi-variate gaussian variables :
	
	tc = np.sum(list(map(lambda x : ent_fun(1,x), single_vars)) - biascorr_1) - (ent_fun(N,emp_det) - biascorrN) #Total correlation
	
	Hred = 0
	
	for red in range(1, N+1):
		Hred += ent_fun((N-1), np.linalg.det(reduce_x(red-1, covmat))) - biascorrNmin1
		
	dtc = Hred - (N-1) * (ent_fun(N, emp_det)-biascorrN) # dtc = Dual Total Correlation

	oinfo = tc - dtc
	sinfo = tc + dtc
	
	return oinfo, sinfo

*example :*

In [26]:
covmat = [[ 1., 2.], [ 3., 4.]]
T = 15


sinfo, oinfo = soinfo_from_covmat (covmat, T)
print('Sinfo = ', sinfo)
print('Oinfo = ', oinfo)

Sinfo =  0j
Oinfo =  (0.6132741758614122-3.141592653589793j)


**high_order()** *compute S-Information, O-Information, and characterize the High Order interactions among n variables governed by Redundancy or Synergy.*

In [27]:
def high_order (data, n):

	""" 
	
	INPUTS :
	
	data = Matrix with dimensionality (N,T), where N is the number of brain regions or 
	modules, and T is the number of samples.
	n = number of interactions or n-plets.
	
	OUTPUTS :
	Red = Matrix with dimension (1, Modules), with the redundancy values per patient 
	and per module.
	Syn = Matrix with dimension (1, Modules), with synergy values per patient 
	and per module.
	Oinfo = O-Information for all the n-plets.
	Sinfo = S-Information for all the n-plets.
	
	"""

	### INITIALISATION :
	
	Modules = len(data)
	Red = np.zeros(Modules) 
	Syn = np.zeros(Modules)
	
	vector = np.arange(0, Modules)
	nplets = []
	
		
	Oinfo = []
	Sinfo = []
	
	
	### N-PLETS CALCULATION :
	
	for x in combinations(vector, n): # n-tuples without repetition over 20 modules
		nplets.append(x)
	nplets = np.array(nplets)

	### DATA NORMALISATION :

	mean = np.mean(data, axis = 1)
	mean = mean.reshape(-1,1)
	
	dataNorm = data - mean

	gaussian_data, cov_mat = data2gaussian (np.transpose(dataNorm)) # Transformation to Copulas and Covariance Matrix Estimation
	
	i = 0
	
	### OINFO AND SINFO COMPUTATION :	

	Info = []
	Info.append(list(map(lambda x : soinfo_from_covmat(cov_mat[np.ix_(x,x)], len(dataNorm[0])), nplets)))
	Info = np.transpose(Info)
	Oinfo = np.array(Info[0])
	Sinfo = np.array(Info[1])
	
	### REDUNDANCY AND SYNERGY COMPUTATION :
	
	"""
	Here, we want to verify in each nplet if a module exist : when it is the case we compute
	the associated Redundancy and Synergy, according to the Oinfo value of these nplets.
	"""
	Values = []
	
	for module in range(Modules):
		Values, cols = np.where(nplets == module)
		
		Oinfo_module_pos = []
		Oinfo_module_neg = []
		
		for i in range(len(Values)):

			if Oinfo[Values[i]].real > 0 :
				Oinfo_module_pos.append(Oinfo[Values[i]].real)
			if Oinfo[Values[i]].real < 0 :
				Oinfo_module_neg.append(Oinfo[Values[i]].real)

		Red[module] = np.mean(np.array(Oinfo_module_pos))
		Syn[module] = np.mean(np.absolute(np.array(Oinfo_module_neg)))
		
		Values = []
		
	Red = Red.reshape(-1,1)
	Syn = Syn.reshape(-1,1)
	
	np.nan_to_num(Syn)
		
	
	return Red, Syn, Oinfo, Sinfo

*example 1 : Toy example*

In [28]:
fil = 'data.txt'
data = np.loadtxt(fil)
data = np.transpose(data)

n = 3

print("Experiment :")
print("Data shape : ", np.shape(data))
print("N-plet : N = ", n)
Red, Syn, Oinfo, Sinfo = high_order (data, n)


print("Red = \n", Red)
print("Syn = \n", Syn)
print("Oinfo = \n", Oinfo)
print("Sinfo = \n", Sinfo)

Experiment :
Data shape :  (5, 10)
N-plet : N =  3
Red = 
 [[0.0093925 ]
 [0.00595465]
 [0.01136318]
 [0.01257646]
 [0.01121641]]
Syn = 
 [[0.04521023]
 [0.0792501 ]
 [0.04348603]
 [0.04607233]
 [0.02819029]]
Oinfo = 
 [[ 0.0036509 +0.j]
 [-0.0792501 +0.j]
 [ 0.0063793 +0.j]
 [ 0.0181473 +0.j]
 [-0.04348603+0.j]
 [-0.01289456+0.j]
 [ 0.00331553+0.j]
 [ 0.00964334+0.j]
 [ 0.00678416+0.j]
 [ 0.02205885+0.j]]
Sinfo = 
 [[-0.11421223+0.j]
 [ 0.2619684 +0.j]
 [-0.14121204+0.j]
 [-0.28507303+0.j]
 [ 0.01545767+0.j]
 [-0.17268191+0.j]
 [-0.18133203+0.j]
 [-0.31477674+0.j]
 [-0.15281791+0.j]
 [-0.24202692+0.j]]


*example 2 : EEG Data example*

In [29]:
fil = 'sub-01_task-seegstim_run-01_epochs.npy'
data = np.load(fil)


n_epoch=data.shape[0] # Number of epochs
n_signal=data.shape[1] # Number of signals
n_point=data.shape[2] # Number voltage values at each epoch

# Extract the signals
# signals: (256,38*2081) matrix
signals=np.zeros((n_signal,n_epoch*n_point))
for i in range (n_signal):
    for j in range (n_epoch):
        signals[i,j*n_point:(j+1)*n_point]=data[j,i,:]

data = np.zeros((20, 150))

for i in range(len(data)):
	for j in range(len(data[i])):
		data[i][j] = signals[i][j]

n = 3

print("Experiment :")
print("Data shape : ", np.shape(data))
print("N-plet : N = ", n)
Red, Syn, Oinfo, Sinfo = high_order (data, n)


print("Red = \n", Red)
print("Syn = \n", Syn)
print("Oinfo = \n", Oinfo)
print("Sinfo = \n", Sinfo)

Experiment :
Data shape :  (20, 150)
N-plet : N =  3
Red = 
 [[0.1016804 ]
 [0.16686132]
 [0.1702303 ]
 [0.14864651]
 [0.17163629]
 [0.16153031]
 [0.13749736]
 [0.08065497]
 [0.03668144]
 [0.15752672]
 [0.1597381 ]
 [0.15708144]
 [0.14333413]
 [0.09416274]
 [0.07987271]
 [0.10106591]
 [0.05822325]
 [0.11747604]
 [0.15068438]
 [0.1589152 ]]
Syn = 
 [[0.01118193]
 [0.0269671 ]
 [0.01564277]
 [0.0110644 ]
 [0.01695087]
 [0.01054038]
 [0.00997889]
 [0.00273669]
 [0.01180196]
 [0.00985562]
 [0.0100757 ]
 [0.00396579]
 [0.01014993]
 [0.00688864]
 [0.00449911]
 [0.01339305]
 [0.00977817]
 [0.01140549]
 [0.00491404]
 [0.01086349]]
Oinfo = 
 [[ 0.25357335+0.j]
 [ 0.19106312+0.j]
 [ 0.24604218+0.j]
 ...
 [-0.01663759+0.j]
 [ 0.03658603+0.j]
 [ 0.22221867+0.j]]
Sinfo = 
 [[1.44442979+0.j]
 [1.36255458+0.j]
 [1.45602463+0.j]
 ...
 [0.69480616+0.j]
 [0.99432633+0.j]
 [1.359193  +0.j]]
